<a href="https://colab.research.google.com/github/macarov13/Analitica-de-negocios/blob/main/2do_modelo_Arbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Arboles de Decisión**
0. Cargar las librerias de referencia para el modelo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #Graficador de cosas
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#Para no tener que cargar el archivo
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Se procede a la carga de datos del archivo

In [ ]:
nxl='/content/drive/MyDrive/Colab Notebooks/0. SolicitantesCrédito(USD) (1).xlsx'
XDB=pd.read_excel(nxl,sheet_name=1)
XDB=XDB.dropna() #Elimina celdas vacias
XDB.head(100)

#Se procede con la selección de las variables
XD=np.array(XDB[['Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']]) #Datos entrada
print(XD)
yd=np.array(XDB[['PreApr']]) #Dato que quiero que el modelo pronostique
print(yd)

[[  38.         1356.144      1685.62266667  625.56223022]
 [  51.          286.016       364.462       140.03198442]
 [  37.          517.46325     629.20888889  284.56449156]
 ...
 [  38.          773.01975     672.91066667  594.94789369]
 [  43.          635.50175     780.69155556  305.58053861]
 [  37.         1419.26675    1169.83844444 1005.63687051]]
[[1]
 [1]
 [0]
 ...
 [1]
 [1]
 [1]]


2. Creación del modelo - Decision tree



In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4) #4 niveles o variables de entrada
mar.fit(XD,yd)
ydp=mar.predict(XD)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(yd,ydp)
print(cm)

#Evaluamos el modelo
VN=cm[0,0]; FP=cm[0,1]; FN=cm[1,0]; VP=cm[1,1]

Ex=(VN+VP)/(VN+VP+FN+FP) #Que porcentaje de los datos pronostico correctamente
print("La exactitud alcanzada por el modelo es:", Ex)

Se=VP/(VP+FN) #Que porcentaje de los datos positivos pronosticados correctamente
print("La sensibilidad alcanzada por el modelo es:", Se)

Es=VN/(VN+FP) #Que porcentaje de los datos negativos pronosticados correctamente
print("La especificidad alcanzada por el modelo es:", Es)

#Este modelo tiene preferencia por los pre-negados

[[2301  658]
 [ 644 2239]]
La exactitud alcanzada por el modelo es: 0.7771311194796303
La sensibilidad alcanzada por el modelo es: 0.7766215747485259
La especificidad alcanzada por el modelo es: 0.7776275768840825


3. ¿Y, donde esta el arbol?

In [ ]:
#Librerias para graficar el árbol
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

vs=['Edad', 'Ingresos', 'Egresos', 'Monto (EAD)'] #Titulos para los nodos
dot_graph=export_graphviz(mar, feature_names=vs)
graph=graph_from_dot_data(dot_graph)
graph.write_png('arbol.png')

NameError: name 'mar' is not defined

Hay 5842 datos.
2959 prenegados.
2883 preaprobados.


4. Se procede con el pronostico de los nuevos solicitantes

In [ ]:
XDB2=pd.read_excel(nxl,sheet_name=2)
XDB2=XDB2.dropna() #Elimina celdas vacias
XDB2.head(100)

XD2=np.array(XDB2[['Edad','Ingresos','Egresos','Monto (EAD)']]) #Datos entrada
print(XD2)

#Pronosticamos la aprobacion
ydp2=mar.predict(XD2)
print(ydp2)

print("El numero de PreAprobados es:", np.sum(ydp2))
print("El numero total de datos es:", len(ydp2))
print("El numero de PreNegados es:", len(ydp2)-np.sum(ydp2))


[[  38.          383.67775     368.59866667  175.84723507]
 [  47.           34.1875       38.03444444   26.95587069]
 [  46.         1774.305      2116.72377778  705.43470796]
 ...
 [  32.          283.26575     170.26644444  225.30034584]
 [  35.          768.288       847.26444444  479.16423193]
 [  24.          487.598       376.18355556  424.283721  ]]
[0 0 1 ... 0 1 0]
El numero de PreAprobados es: 1260
El numero total de datos es: 2536
El numero de PreNegados es: 1276


**Analisis de resultados**

De acuerdo con la base de datos suministradas y el nodo base del arbol de decision se puede observar un total de 5842 datos, divididos en 2959 PreAprobados y un total de 2883 PreNegados, esto representa el 51% y el 49% respectivamente. De acuerdo con el pronostico realizado por el modelo, este alncanzó una exactitud cercana al 77%, valores que fueron similares a la sensibilidad que mide el comportamiento del modelo frente a la clasificación de los PreAprobados, asi como frente a la especificidad indicador que mide el comportamiento del modelo frente a la clasificacion de los PreNegados.

Se destaca el unico nodo puro "(PreNeg:10, PreApro: 0)", en donde los solicitantes de credito deben cumplir con la siguiente regla: SI Ing>644 AND Egr<=685 AND Ing>376 AND Monto>=322.

Con respecto al pronostico, se puede observar un total de 2536 individuos sin PreAprobacion, de los cuales un total de 1260 pueden ser PreAprobados y un total de 1276 que pueden ser PreNegados.